In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip -qq "/content/gdrive/MyDrive/dacon 이미지 (1).zip"

## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#WeightedRandomSampler는 PyTorch에서 제공하는 샘플링 방식 중 하나로, 클래스 불균형이 있는 데이터셋에서 사용할 수 있습니다. 클래스별로 다른 가중치를 부여하여 데이터를 샘플링하는 방식
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

#이미지 합치기

#녹오염

In [ ]:
# import os
# import cv2
# import random
# import numpy as np

# # 이미지가 저장된 경로
# image_dir = "/content/train/녹오염"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)



# import os
# import cv2
# import random
# import numpy as np

# # 이미지가 저장된 경로
# image_dir = "/content/train/면불량"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/석고수정"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)

# # 이미지가 저장된 경로
# image_dir = "/content/train/들뜸"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기


# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/피스"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/창틀,문틀수정"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/피스"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/창틀,문틀수정"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)



# # 이미지가 저장된 경로
# image_dir = "/content/train/이음부불량"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/창틀,문틀수정"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)

In [ ]:
# # 이미지가 저장된 경로
# image_dir = "/content/train/가구수정"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)


# # 이미지가 저장된 경로
# image_dir = "/content/train/반점"

# # 이미지 불러오기
# images = []
# for file in os.listdir(image_dir):
#     if file.endswith(".jpg") or file.endswith(".png"):
#         image = cv2.imread(os.path.join(image_dir, file))
#         images.append(cv2.resize(image, (224, 224))) # 이미지 크기 맞추기

# # 이미지 합치기 및 저장
# for i in range(len(images)//2):
#     # 이미지 랜덤 선택
#     idx1, idx2 = random.sample(range(len(images)), 2)
#     image1 = images[idx1]
#     image2 = images[idx2]

#     # 이미지 합치기
#     merged_image = np.hstack((image1, image2))

#     # 합쳐진 이미지 저장
#     cv2.imwrite(os.path.join(image_dir, f"merged_{i}.jpg"), merged_image)



# Data Pre-processing

In [ ]:

all_img_list = glob.glob('./train/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[2])

In [ ]:

df

,img_path,label
0,./train/곰팡이/81.png,곰팡이
1,./train/곰팡이/0.png,곰팡이
2,./train/곰팡이/50.png,곰팡이
3,./train/곰팡이/14.png,곰팡이
4,./train/곰팡이/67.png,곰팡이
...,...,...
3452,./train/몰딩수정/1.png,몰딩수정
3453,./train/몰딩수정/86.png,몰딩수정
3454,./train/몰딩수정/122.png,몰딩수정
3455,./train/몰딩수정/115.png,몰딩수정


In [ ]:

counts = df['label'].value_counts()

In [ ]:
counts

훼손         1405
오염          595
걸레받이수정      307
꼬임          210
터짐          162
곰팡이         145
오타공         142
몰딩수정        130
면불량          99
석고수정         57
들뜸           54
피스           51
창틀,문틀수정      27
울음           22
이음부불량        17
녹오염          14
가구수정         12
틈새과다          5
반점            3
Name: label, dtype: int64

In [ ]:

train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

## CustomDataset

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Flip(p=0.4),
    #A.GaussNoise(p=0.1),
    #A.ElasticTransform(p=0.1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), always_apply=True),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Flip(p=0.4),
    #A.GaussNoise(p=0.1),
    #A.ElasticTransform(p=0.1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), always_apply=True),
    ToTensorV2(),
])


In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [ ]:

#efficientnet_b0
# class BaseModel(nn.Module):
#     def __init__(self, num_classes=len(le.classes_)):
#         super(BaseModel, self).__init__()
#         self.backbone = models.efficientnet_b1(pretrained=True)
#         self.classifier = nn.Linear(1000, num_classes)
        
#     def forward(self, x):
#         x = self.backbone(x)
#         x = self.classifier(x)
#         return x

In [ ]:

#efficientnet_b1
# class BaseModel(nn.Module):
#     def __init__(self, num_classes=len(le.classes_)):
#         super(BaseModel, self).__init__()
#         self.backbone = models.efficientnet_b1(pretrained=True)
#         self.classifier = nn.Linear(1000, num_classes)
        
#     def forward(self, x):
#         x = self.backbone(x)
#         x = self.classifier(x)
#         return x

In [ ]:

# class BaseModel(nn.Module):
#     def __init__(self, num_classes=len(le.classes_)):
#         super(BaseModel, self).__init__()
#         self.backbone = models.resnet18(pretrained=True)
#         self.classifier = nn.Linear(1000, num_classes)
        
#     def forward(self, x):
#         x = self.backbone(x)
#         x = self.classifier(x)
#         return x

In [ ]:
from torchvision.models import efficientnet_b6 as efficientnet_b6

In [ ]:
from torchvision.models import densenet201 as densenet201

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.densenet201(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [ ]:
model = BaseModel()
model.eval()
#optimizer = torch.optim.RMSprop(model.parameters(), lr=CFG["LEARNING_RATE"])
optimizer = optim.Adagrad(model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)
#infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 262MB/s]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.26580] Val Loss : [1.01081] Val Weighted F1 Score : [0.67581]


  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.17492] Val Loss : [1.12747] Val Weighted F1 Score : [0.76157]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.13180] Val Loss : [1.27084] Val Weighted F1 Score : [0.74395]


epoch10에서 0.78이 나옴. 이게 제일 베스트
Epoch [10], Train Loss : [0.01444] Val Loss : [0.78749] Val Weighted F1 Score : [0.81940]

p=0.4 Epoch [10], Train Loss : [0.02424] Val Loss : [0.79207] Val Weighted F1 Score : [0.82431]----제일 베스트

Epoch [10], Train Loss : [0.01979] Val Loss : [0.65254] Val Weighted F1 Score : [0.83970]--- 어차피 같은 이미지로 인식해서 아마 변별력이 없는듯.

## Inference

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit.to_csv('./densenet201_base.csv', index=False)

In [ ]:
import torch

# 모델 저장
torch.save(model.state_dict(), "model.pt")

# 가중치 저장
torch.save(optimizer.state_dict(), "optimizer.pt")